In [1]:
from custom_clss_and_fncs import *
from cobra.io import load_json_model

model                                 = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
feature_data = pd.read_parquet("./results/data/oversampled_augmented_metabolite_data.parquet.gzip")#
feature_names                         = pd.read_csv("./results/data/metabolite_names.csv")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### add concentratios as features

In [14]:
grafo_nx                              = cobra_a_networkx(model)

node_list = list(grafo_nx.nodes)
feature_data.rename(
    columns=feature_names.set_index("Simbolo_traductor")["Recon3_ID"].to_dict(), 
    inplace=True
)

number_to_fill_in = int(1)

feature_data[
    [item for item in node_list if item not in feature_data.columns.tolist()]
] = number_to_fill_in

# Re-empaca todo este vector como un diccionario de atributos
feature_dict = feature_data.to_dict(orient="list")  # desde Pandas
feature_dict = {key: {"x": feature_dict[key]} for key in feature_dict}

nx.set_node_attributes(grafo_nx, feature_dict)

assert feature_data['phe_L_c'].tolist() == grafo_nx.nodes['phe_L_c']['x']
assert np.unique(grafo_nx.nodes(data=True)['r0399']['x']) == number_to_fill_in
assert len(grafo_nx.nodes(data=True)['r0399']['x']) == len(grafo_nx.nodes(data=True)['phe_L_c']['x'])

In [15]:
nx.write_gpickle(grafo_nx, f"./results/graphs_from_PYG_and_NX/NX_only_concentrations.gpickle")

In [16]:
grafo_nx_v2 = copy.deepcopy(grafo_nx)

### add fluxes as features

In [18]:
flux_samples = pd.read_parquet( "./results/data/flux_samples.parquet.gzip")

In [29]:
feature_length = len(grafo_nx.nodes(data=True)['r0399']['x']) 


flux_dict = flux_samples.sample(feature_length, replace=True).to_dict(orient = 'list')

In [39]:
x_attribute     = nx.get_node_attributes(grafo_nx_v2, "x")

x_attribute.update(flux_dict)

nx.set_node_attributes(grafo_nx_v2, x_attribute, 'x')


assert grafo_nx_v2.nodes(data=True)['r0399']['x'] == flux_dict['r0399']
assert len(grafo_nx_v2.nodes(data=True)['r0399']['x']) == len(grafo_nx_v2.nodes(data=True)['phe_L_c']['x'])
assert grafo_nx_v2.nodes(data=True)['phe_L_c']['x']  == feature_data['phe_L_c'].tolist() 

In [40]:
nx.write_gpickle(grafo_nx_v2, f"./results/graphs_from_PYG_and_NX/NX_fluxes_and_concentrations.gpickle")